In [1]:
import torch
import torch.nn as nn

class CustomLayer(nn.Module):
    def __init__(self, n_features):
        super(CustomLayer, self).__init__()
        self.n_features = n_features
        # Initialize the interaction tensor as a learnable parameter
        self.interaction_tensor = nn.Parameter(torch.randn(n_features, n_features, n_features))

    def forward(self, x):
        # x is expected to be of size [batch, length, n_features]
        batch, length, n_features = x.size()

        # Process each [n_features] vector across batch and length
        output = x.new_empty(batch, length, n_features)
        for b in range(batch):
            for l in range(length):
                state_tensor = x[b, l, :]  # Shape: [n_features]
                # Step 1: Multiply state tensor by interaction tensor to get transition tensor
                # We manually implement the multiplication to match your operation
                transition_tensor = torch.einsum('i,ijk->jk', state_tensor, self.interaction_tensor)
                # Step 2: Multiply the transition tensor by the state tensor
                # Resulting shape: [n_features]
                output[b, l, :] = torch.matmul(transition_tensor, state_tensor)

        return output


In [2]:
import torch
import torch.nn as nn

class CustomLayerVectorized(nn.Module):
    def __init__(self, n_features):
        super(CustomLayerVectorized, self).__init__()
        self.n_features = n_features
        # Initialize the interaction tensor as a learnable parameter
        self.interaction_tensor = nn.Parameter(torch.randn(n_features, n_features, n_features))

    def forward(self, x):
        # x is expected to be of size [batch, length, n_features]
        batch, length, n_features = x.size()

        # Pre-allocate output tensor
        output = x.new_empty(batch, length, n_features)

        # Loop over length, but vectorize over the batch
        for l in range(length):
            # Extract all vectors at position l across all batches
            state_tensor = x[:, l, :]  # Shape: [batch, n_features]
            
            # Vectorized operation for all batches
            # Step 1: Calculate the transition tensor
            # Since we cannot directly use einsum for batched operation in this specific scenario,
            # we manually broadcast and multiply to achieve the intended result.
            # This involves expanding dimensions to enable broadcasting.
            state_tensor_expanded = state_tensor.unsqueeze(1).expand(-1, n_features, -1)  # Shape: [batch, n_features, n_features]
            interaction_tensor_expanded = self.interaction_tensor.unsqueeze(0).expand(batch, -1, -1, -1)  # Shape: [batch, n_features, n_features, n_features]
            # Multiply and sum over the last dimension to get the transition tensor
            transition_tensor = torch.einsum('bik,bijk->bij', state_tensor_expanded, interaction_tensor_expanded)
            
            # Step 2: Multiply the transition tensor by the state tensor to get the output
            output[:, l, :] = torch.einsum('bij,bj->bi', transition_tensor, state_tensor)

        return output



In [3]:
# n_features = 16
# batch, length, dim = 2, 64, n_features
# x = torch.randn(batch, length, dim)
# model = CustomLayerVectorized(
#     n_features = n_features
# )
# y = model(x)

# print(x.shape)
# assert y.shape == x.shape

In [4]:
import torch

def torch_randnorm(size, dim=0):
    # Generate a random tensor
    rand_tensor = torch.rand(size)
    
    # Normalize along the specified dimension
    sum_along_dim = torch.sum(rand_tensor, dim=dim, keepdim=True)
    normalized_tensor = rand_tensor / sum_along_dim
    
    return normalized_tensor

# Example usage
normalized_tensor = torch_randnorm([5,6], dim=1)
print(normalized_tensor)
print(normalized_tensor.sum(dim=1))

tensor([[0.0882, 0.0566, 0.4550, 0.0727, 0.1311, 0.1963],
        [0.2407, 0.1104, 0.2277, 0.2421, 0.1738, 0.0053],
        [0.1226, 0.0789, 0.1218, 0.1194, 0.2616, 0.2957],
        [0.2379, 0.0828, 0.2108, 0.0702, 0.2090, 0.1894],
        [0.0027, 0.3202, 0.2005, 0.2144, 0.0303, 0.2318]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [5]:
class InteractionModule(nn.Module):
    def __init__(self, n_features):
        super(InteractionModule, self).__init__()
        self.n_features = n_features
        # Initialize a set of interaction tensors, one for the state tensor and one for each column of the transition tensor
        self.interaction_tensors = nn.ParameterList([nn.Parameter(torch_randnorm([n_features, n_features, n_features], dim = 1)) for _ in range(n_features + 1)])

    def forward(self, state_tensor, previous_transition_tensor):
        # Get batch size
        batch = state_tensor.shape[0]
        # Assuming previous_transition_tensors is a list of transition tensors from the previous step
        candidates = []
        for i in range(self.n_features + 1):
            if i == 0:  # Interaction with the state tensor
                current_tensor = state_tensor
            else:  # Interaction with columns of the previous transition tensor
                current_tensor = previous_transition_tensor[:, :, i - 1]

            interaction_tensor = self.interaction_tensors[i]
            current_tensor_expanded = current_tensor.unsqueeze(1).expand(-1, self.n_features, -1)  # Shape: [batch, n_features, n_features]
            interaction_tensor_expanded = interaction_tensor.unsqueeze(0).expand(batch, -1, -1, -1)  # Shape: [batch, n_features, n_features, n_features]
            # Multiply and sum over the last dimension to get the transition tensor
            candidate = torch.einsum('bik,bijk->bij', current_tensor_expanded, interaction_tensor_expanded)
            
            candidates.append(candidate)
            
        candidates_tensor = torch.stack(candidates, dim = -1)
        return candidates_tensor


In [6]:
class CustomLayerExtended(nn.Module):
    def __init__(self, n_features):
        super(CustomLayerExtended, self).__init__()
        self.n_features = n_features
        self.interaction_module = InteractionModule(n_features)
        self.selector_module = SelectorModule(n_features + 1)

    def forward(self, x):
        batch, length, n_features = x.size()
        output = x.new_empty(batch, length, n_features)

        # Initialize previous transition tensors (for the first step)
        # Assuming it's a list of zero tensors for simplicity
        previous_transition_tensor = torch.zeros(batch, n_features, n_features)

        for l in range(length):
            state_tensor = x[:, l, :]
            # Generate candidates
            candidates = self.interaction_module(state_tensor, previous_transition_tensor)
            # Select one candidate
            selected_transition_tensor = self.selector_module(candidates)
            # Update the previous_transition_tensors for the next iteration
            previous_transition_tensor = selected_transition_tensor
            # Compute output for this step
            output[:, l, :] = torch.matmul(selected_transition_tensor, state_tensor.unsqueeze(-1)).squeeze(-1)

        return output, selected_transition_tensor


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelectorModule(nn.Module):
    def __init__(self, num_slices):
        super(SelectorModule, self).__init__()
        # A simple linear layer to compute importance scores for each slice
        self.importance = nn.Linear(num_slices, num_slices)

    def forward(self, x):
        # Assuming x is of shape [batch_size, height, width, num_slices]
        batch_size, _, _, num_slices = x.shape

        # Compute importance scores by reducing x across spatial dimensions
        # Here, we take the mean of x across the spatial dimensions to get a vector per slice
        x_reduced = x.mean(dim=[1, 2])  # Shape: [batch_size, num_slices]

        # Compute scores for each slice
        scores = self.importance(x_reduced)  # Shape: [batch_size, num_slices]

        # Apply softmax to get weights for each slice
        weights = F.softmax(scores, dim=-1)  # Shape: [batch_size, num_slices]

        # Correct application of weights:
        # We need to ensure weights are applied across the num_slices dimension correctly.
        # Since weights are [batch_size, num_slices] and x is [batch_size, height, width, num_slices],
        # we permute x to bring num_slices to the front for broadcasting.
        x_permuted = x.permute(0, 3, 1, 2)  # Shape: [batch_size, num_slices, height, width]
        
        # Now, multiply by weights. We need to reshape weights to [batch_size, num_slices, 1, 1] for broadcasting.
        weighted_slices = x_permuted * weights.view(batch_size, num_slices, 1, 1)
        
        # Finally, sum the weighted slices across the num_slices dimension (now the first dimension after permute)
        selected = weighted_slices.sum(dim=1)  # Shape: [batch_size, height, width]

        return selected

# Example usage
batch_size = 10
tensor = torch.rand(batch_size, 5, 5, 6)  # Example tensor
model = SelectorModule(num_slices=6)

result = model(tensor)
print(result.shape)  # Should print torch.Size([10, 5, 5])


torch.Size([10, 5, 5])


In [8]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

class EnhancedModelWithMHA(nn.Module):
    def __init__(self, n_features, num_heads):
        super(EnhancedModelWithMHA, self).__init__()
        self.pos_encoder = PositionalEncoding(n_features)
        self.multihead_attn = nn.MultiheadAttention(embed_dim=n_features, num_heads=num_heads)
        self.feed_forward = nn.Linear(n_features, n_features)
        
    def forward(self, x):
        x = x.permute(1, 0, 2)  # Adjust for MHA: [seq_len, batch, features]
        x = self.pos_encoder(x)
        attn_output, _ = self.multihead_attn(x, x, x)
        output = F.relu(self.feed_forward(attn_output))
        output = output.permute(1, 0, 2)  # Adjust back: [batch, seq_len, features]
        return output


# Example Training

## Simple Data Generation

In [9]:
import numpy as np
import torch

def generate_normalized_multivariate_time_series(n_features, total_length, amplitude=1.0):
    t = np.linspace(0, 100 * np.pi, total_length)
    series = np.zeros((total_length, n_features))
    for i in range(n_features):
        series[:, i] = amplitude * np.cos(t * (i + 1) / n_features) + 10
    
    # Normalize such that each timestep's values sum to 1
    series_sum = np.sum(series, axis=1, keepdims=True)
    series_normalized = series / series_sum
    
    return series_normalized

def segment_time_series(series, length):
    # Assuming series is a numpy array of shape [total_length, n_features]
    total_length, n_features = series.shape
    segments = []
    for start in range(0, total_length - length, length):
        segment = series[start:start + length]
        segments.append(segment)
    return np.stack(segments)


In [10]:
n_features = 4
length = 640  # Segment length
total_length = 10240  # Arbitrary total length for the generated series

# Generate and segment the time series
series = generate_normalized_multivariate_time_series(n_features, total_length)
series_x = series[:-1,]
series_y = series[1:,]

segments_x = segment_time_series(series_x, length)
segments_y = segment_time_series(series_y, length)

# Convert to tensors
segments_tensor_x = torch.tensor(segments_x, dtype=torch.float)
segments_tensor_y = torch.tensor(segments_y, dtype=torch.float)

# Prepare inputs and targets
X = segments_tensor_x
# Shift segments to the right by one timestep to create the targets
Y =  segments_tensor_y


In [11]:
from torch import nn, optim

# Model
num_heads = 4
model = EnhancedModelWithMHA(n_features=n_features, num_heads=num_heads)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


Epoch [10/100], Loss: 0.03248867020010948
Epoch [20/100], Loss: 0.03218349441885948
Epoch [30/100], Loss: 0.03173460811376572
Epoch [40/100], Loss: 0.03148769214749336
Epoch [50/100], Loss: 0.031460363417863846
Epoch [60/100], Loss: 0.03146583214402199
Epoch [70/100], Loss: 0.03146149963140488
Epoch [80/100], Loss: 0.031457945704460144
Epoch [90/100], Loss: 0.03145666792988777
Epoch [100/100], Loss: 0.03145628795027733


In [12]:
torch.stack([X[1,:,1],Y[1,:,1]], dim = -1)

tensor([[0.2302, 0.2306],
        [0.2306, 0.2309],
        [0.2309, 0.2312],
        ...,
        [0.2722, 0.2720],
        [0.2720, 0.2719],
        [0.2719, 0.2717]])

In [13]:
import plotly.graph_objects as go

def plot_model_output_vs_target(model_outputs, targets, batch_index=0, feature_index=0):
    # Extract the specified feature for the given batch from both the model outputs and targets
    model_output_series = model_outputs[batch_index, :, feature_index].detach().numpy()
    target_series = targets[batch_index, :, feature_index].numpy()
    
    # Create a range for the x-axis (timesteps)
    timesteps = list(range(model_output_series.shape[0]))
    
    # Create traces
    model_trace = go.Scatter(x=timesteps, y=model_output_series, mode='lines', name='Model Output')
    target_trace = go.Scatter(x=timesteps, y=target_series, mode='lines', name='Target')
    
    # Create the figure and add traces
    fig = go.Figure()
    fig.add_trace(model_trace)
    fig.add_trace(target_trace)
    
    # Add title and labels
    fig.update_layout(title=f'Model Output vs Target for Feature {feature_index}, Batch {batch_index}',
                      xaxis_title='Timestep',
                      yaxis_title='Value')
    
    # Show the figure
    fig.show()

# Assuming `y` and `Y` are your model outputs and targets, respectively
# Adjust batch_index and feature_index as needed
plot_model_output_vs_target(outputs, Y, batch_index=2, feature_index=3)


In [14]:
torch.stack([outputs[1,:,1],Y[1,:,1]], dim = -1)

tensor([[0.2491, 0.2306],
        [0.2495, 0.2309],
        [0.2500, 0.2312],
        ...,
        [0.2500, 0.2720],
        [0.2496, 0.2719],
        [0.2492, 0.2717]], grad_fn=<StackBackward0>)